In [1]:
import pandas as pd
import numpy as np


In [ ]:

import regex as re
import nltk
def tokensize(text):
    return re.findall(r'[\w-]*\p{L}[\w-]*',text)

nltk.download("all")


In [3]:
un = pd.read_csv("/content/drive/MyDrive/UN/un-general-debates.csv")


In [4]:
stopwords = set(nltk.corpus.stopwords.words('english'))

def remove_stop(tokens):
    return [t for t in tokens if t.lower() not in stopwords]
     

pipeline = [str.lower,tokensize,remove_stop]

def prepare(text, pipeline):
    tokens = text
    for transform in pipeline:
        tokens = transform(tokens)
    return tokens
     

un['tokens'] = un['text'].apply(prepare, pipeline=pipeline)
un['num_tokens'] = un['tokens'].map(len)        

# 컨텍스트 내 키워드 탐색

>워드 클라우드와 빈도 다이어그램은 텍스트 데이터를 시각적으로 요약하는 훌륭한 도구지만 특정 용어가 왜 그렇게 두드러지게 등장하는지는 의문을 제기하기도 한다.

<br>

실제로, 내용을 이해하려면 전처리되지 않은 원본 텍스트에서 해당 단어가 사용된 곳을 직접 찾아 확인해야하는데 분량이 많을 것이다.

이를 영리하게 처리하는 방법은 컨텍스트 내 키워드(KWIC) 분석법이다.

이는, 키워드를 중심으로 왼쪽 및 오른쪽에 있는 일정한 개수의 단어를 목록화한다.

## KWIC 분석

- NLTK 및 textacy로 구현된다

- textacy의 KWIC함수를 사용하고 이 함수는 토큰화되지 않은 텍스트에서 잘 작동하며 빠르게 실행되고, climate change와 같이 여러 토큰에 걸쳐 있는 문자열을 검색할 수 있다.(NLTK는 검색할 수 없다.) NLTK와 textacy의 KWIC함수는 모두 단일 문서에서만 작동하고, 데이터프레임의 여러 문서에서 해석이 가능하도록 확장하겠다.

In [ ]:
!pip install textacy


In [8]:
import textacy

if textacy.__version__ < '0.11': # as in printed book
    from textacy.text_utils import KWIC
    
else: # for textacy 0.11.x
    from textacy.extract.kwic import keyword_in_context

    def KWIC(*args, **kwargs):
        # call keyword_in_context with all params except 'print_only'
        return keyword_in_context(*args, 
                           **{kw: arg for kw, arg in kwargs.items() 
                            if kw != 'print_only'})

In [14]:
from tqdm import tqdm
def kwic(doc_series, keyword, window=35, print_samples=5):

    def add_kwic(text):
        kwic_list.extend(KWIC(text, keyword, ignore_case=True, 
                              window_width=window, print_only=False))

    kwic_list = []
    doc_series.progress_map(add_kwic)

    if print_samples is None or print_samples==0:
        return kwic_list
    else:
        k = min(print_samples, len(kwic_list))
        print(f"{k} random samples out of {len(kwic_list)} " + \
              f"contexts for '{keyword}':")
        for sample in random.sample(list(kwic_list), k):
            print(re.sub(r'[\n\t]', ' ', sample[0])+'  '+ \
                  sample[1]+'  '+\
                  re.sub(r'[\n\t]', ' ', sample[2]))

위의 함수는 map를 이용해서 각 문서에 add_kwic함수를 적용하고, 키워드 컨텍스트를 반복적으로 수집하여서 단어 빈도 분석에서 이미 사용한 이 빕버은 매우 효율적이므로 더 큰 말뭉치에서도 수행할 수 있다.

<br>

**이 함수는 (왼쪽 컨텍스트, 키워드, 오른쪽 컨텍스트)형태의 튜플 리스트를 반환한다.**

>print_sample가 0보다 더 크면 무작위로 샘플링한 결과를 출력하고 샘플링은 주로 목록의 첫째 항목이 단일 또는 매우 적은 수의 문서를 추출해 진행하기 때문에 특히 많은 뭄문서 작업을 할 때 유용하다.

In [ ]:

kwic(un[un['year'] == 2015]['text'], 'sdgs', print_samples=5)